# Preferencias musicales de las ciudades de Springfield y Shelbyville
En este proyecto, compararé las preferencias musicales de las ciudades de Springfield y Shelbyville. Estudiaré datos reales de transmisión de música online para probar la hipótesis y comparar el comportamiento de los usuarios y las usuarias de estas dos ciudades.

Hipótesis: La actividad de los usuarios y las usuarias difiere según el día de la semana y dependiendo de la ciudad.

El proyecto se divide en tres etapas: 
- Etapa 1: Descripción de los datos.
- Etapa 2: Preprocesamiento de datos.
- Etapa 3: Prueba de hipótesis.

Descripción de las columnas:
- 'userID': identifica de forma exclusiva a cada usuario o usuaria;
- 'Track': título de la canción;
- 'artist': nombre del artista;
- 'genre': género musical;
- 'City': ciudad del usuario o la usuaria;
- 'time': hora del día en la que se reprodujo la pista (HH:MM:SS);
- 'Day': día de la semana.

# Etapa 1
Descripción de los datos

Se importan las librerías que se van a utilizar:

In [1]:
import pandas as pd 

Se lee y examina el archivo que contiene los datos: 

In [2]:
df = pd.read_csv('/datasets/music_project_en.csv') # Leer el archivo y almacenarlo en df

In [4]:
df.info() # Obtener la información general de los datos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63736 non-null  object
 2   artist    57512 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


La tabla contiene siete columnas. Hay valores ausentes en 'Track', 'artist' y 'genre'. Almacenan los mismos tipos de datos: object.

Se pueden ver tres problemas con el estilo en los encabezados de la tabla:
1. Algunos encabezados están en mayúsculas, otros en minúsculas.
2. Hay espacios en algunos encabezados.
3. userID no está en formato snake_case.

# Etapa 2
Preparar los datos para que sean analizados

Cambio de los encabezados de la tabla de acuerdo con las reglas del buen estilo:

In [9]:
#Utilizo el bucle for para iterar sobre los nombres de las columnas
new_col_names = []

for old_name in df.columns:
    name_lowered = old_name.lower() #Todos los caracteres en minúsculas
    name_stripped  = name_lowered.strip() #Elimina los espacios al principio y al final de los nombres
    new_col_names.append(name_stripped)

df.columns = new_col_names

In [10]:
#Aplicar la regla de snake_case a la columna userid
columns_new={'userid':'user_id'} 

df.rename(columns=columns_new, inplace=True)

In [11]:
#Resultado final con los nombre de las columnas corregidos
df.columns 

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

Los valores ausentes en 'track' y 'artist' no son cruciales. Así que se van a reemplazar por 'unknown'. 

Pero los valores ausentes en 'genre' pueden afectar la comparación entre las preferencias musicales de Springfield y Shelbyville, sería útil saber las razones por las cuales hay datos ausentes e intentar recuperarlos. Sin embargo, no se tiene esa posibilidad en este este proyecto. Por esto se van a rellenar estos valores ausentes con el valor predeterminado 'unknown'.

Reemplazar los valores ausentes en las columnas 'track', 'artist' y 'genre' con el string 'unknown'. Como mostramos anteriormente en las lecciones, la mejor forma de hacerlo es crear una lista que almacene los nombres de las columnas donde se necesita el reemplazo. Luego, utiliza esta lista e itera sobre las columnas donde se necesita el reemplazo haciendo el propio reemplazo.

Reemplazo los valores ausentes en las columnas 'track', 'artist' y 'genre' con el string 'unknown':

In [18]:
#Creo una lista que almacene los nombres de las columnas donde se necesita el reemplazo
columns_to_replace = ['track', 'artist', 'genre']

# Bucle en los encabezados reemplazando los valores ausentes con 'unknown'
for col in columns_to_replace:
    df[col].fillna('unknown', inplace = True)
    
#Verifico que no haya valores ausentes
df.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

In [19]:
#Cuento duplicados explícitos
df.duplicated().sum()

0

In [23]:
#Elimino duplicados explícitos
df.drop_duplicates(inplace=True)
df.duplicated().sum()

0

Ahora quiero deshacerme de los duplicados implícitos en la columna 'genre'. Por ejemplo, el nombre de un género se puede escribir de varias formas. Dichos errores también pueden afectar al resultado.

Para hacerlo, primero muestro una lista de nombres de género únicos, ordenados en orden alfabético:

In [24]:
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'ambient', 'americana', 'animated', 'anime',
       'arabesk', 'arabic', 'arena', 'argentinetango', 'art', 'audiobook',
       'avantgarde', 'axé', 'baile', 'balkan', 'beats', 'bigroom',
       'black', 'bluegrass', 'blues', 'bollywood', 'bossa', 'brazilian',
       'breakbeat', 'breaks', 'broadway', 'cantautori', 'cantopop',
       'canzone', 'caribbean', 'caucasian', 'celtic', 'chamber',
       'children', 'chill', 'chinese', 'choral', 'christian', 'christmas',
       'classical', 'classicmetal', 'club', 'colombian', 'comedy',
       'conjazz', 'contemporary', 'country', 'cuban', 'dance',
       'dancehall', 'dancepop', 'dark', 'death', 'deep', 'deutschrock',
       'deutschspr', 'dirty', 'disco', 'dnb', 'documentary', 'downbeat',
       'downtempo', 'drum', 'dub', 'dubstep', 'eastern', 'easy',
       'electronic', 'electropop', 'emo', 'entehno', 'epicmetal',
       'estrada', 'ethnic', 'eurofo

Se observan los duplicados implícitos: hip, hop, hip-hop. Así que los voy a llamar a todos hiphop.

In [26]:
#Creo una función con dos parámetros
def replace_wrong_genres(wrong_genres, correct_genre):
    #Itero sobre la lista de géneros incorrectos
    for wrong_genre in wrong_genres: 
        df['genre'].replace(wrong_genre,correct_genre, inplace=True)
    return df

#Elimino los duplicados implícitos
wrong_genre = ['hip', 'hop', 'hip-hop']
correct_genre = 'hiphop'
replace_wrong_genres(wrong_genre, correct_genre)

#Compruebo que no haya duplicados implícitos
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'ambient', 'americana', 'animated', 'anime',
       'arabesk', 'arabic', 'arena', 'argentinetango', 'art', 'audiobook',
       'avantgarde', 'axé', 'baile', 'balkan', 'beats', 'bigroom',
       'black', 'bluegrass', 'blues', 'bollywood', 'bossa', 'brazilian',
       'breakbeat', 'breaks', 'broadway', 'cantautori', 'cantopop',
       'canzone', 'caribbean', 'caucasian', 'celtic', 'chamber',
       'children', 'chill', 'chinese', 'choral', 'christian', 'christmas',
       'classical', 'classicmetal', 'club', 'colombian', 'comedy',
       'conjazz', 'contemporary', 'country', 'cuban', 'dance',
       'dancehall', 'dancepop', 'dark', 'death', 'deep', 'deutschrock',
       'deutschspr', 'dirty', 'disco', 'dnb', 'documentary', 'downbeat',
       'downtempo', 'drum', 'dub', 'dubstep', 'eastern', 'easy',
       'electronic', 'electropop', 'emo', 'entehno', 'epicmetal',
       'estrada', 'ethnic', 'eurofo

# Etapa 3
Prueba de hipótesis 

La hipótesis consiste en comparar el comportamiento de los usuarios en las dos ciudades.

La hipótesis afirma que existen diferencias en la forma en que los usuarios y las usuarias de Springfield y Shelbyville consumen música. Para comprobar esto, voy a utilizar los datos de tres días de la semana: lunes, miércoles y viernes.

In [27]:
#Agrupo los usuarios por ciudad y los cuento
by_city = df.groupby('city')['user_id'].count()
by_city

city
Shelbyville    18512
Springfield    42741
Name: user_id, dtype: int64

Ahora sabemos que en Springfield escuchan 230% más música que en Shelbyville.

In [28]:
#Agrupo los datos por día de la semana y calculo las canciones reproducidas en cada uno de los tres días
by_day = df.groupby('day')['user_id'].count()
by_day

day
Friday       21840
Monday       21354
Wednesday    18059
Name: user_id, dtype: int64

El viernes es el días de la semana que más música escuchan los usuarios y los miércoles el día que menos.

In [29]:
#Creo una función para calcular el número de canciones reproducidas en un determinado día y ciudad
def number_tracks(day, city): 
    #Almaceno las filas del DataFrame donde el valor en la columna 'day' es igual al parámetro day=
    day_filtered = df[df['day'] == day] 
    #Filtro las filas donde el valor en la columna 'city' es igual al parámetro city=
    city_filtered = day_filtered[day_filtered['city'] == city] 
    #Extraigo la columna 'user_id' de la tabla filtrada y aplico el método count()
    users_filtered = city_filtered['user_id'].count() 
    #Devuelvo el número de valores de la columna 'user_id'
    return(users_filtered) 

Voy a llamar a number_tracks(), cambiando los valores de los parámetros para recuperar los datos de ambas ciudades para cada uno de los tres días:

In [30]:
#Canciones reproducidas en Springfield el lunes 
day = 'Monday'
city = 'Springfield' 
df_filtered = number_tracks(day, city) 
df_filtered

15740

In [32]:
#Canciones reproducidas en Shelbyville el lunes 
day = 'Monday'
city = 'Shelbyville' 
df_filtered = number_tracks(day, city) 
df_filtered

5614

In [33]:
#Canciones reproducidas en Springfield el miércoles 
day = 'Wednesday'
city = 'Springfield' 
df_filtered = number_tracks(day, city) 
df_filtered


11056

In [34]:
#Canciones reproducidas en Shelbyville el miércoles 
day = 'Wednesday'
city = 'Shelbyville' 
df_filtered = number_tracks(day, city) 
df_filtered

7003

In [35]:
#Canciones reproducidas en Springfield el viernes 
day = 'Friday'
city = 'Springfield' 
df_filtered = number_tracks(day, city) 
df_filtered

15945

In [36]:
#Canciones reproducidas en Shelbyville el viernes 
day = 'Friday'
city = 'Shelbyville' 
df_filtered = number_tracks(day, city) 
df_filtered

5895

# Conclusión

La hipótesis "La actividad de los usuarios y las usuarias difiere según el día de la semana y dependiendo de la ciudad" es correcta. 

La actividad de los usuarios varia mucho entre una ciudad y la otra, como mencioné anteriormente en Springfield la actividad es 230% más alta que en Shelbyville. Respecto a los días analizados (lunes, miércoles y viernes) también se observa que la actividad varia entre estos.

En ambas ciudades se observa que la actividad los lunes y viernes es muy similar. Sin embargo, en Shelbyville la actividad es mayor los miércoles y en Springfield es menor.